In [1]:
template_string = """You are a master branding consulatant who specializes in naming brands. \
You come up with catchy and memorable brand names.

Take the brand description below delimited by triple backticks and use it to create the name for a brand.

brand description: ```{brand_description}```

then based on the description and you hot new brand name give the brand a score 1-10 for how likely it is to succeed.

Format the output as JSON with the following keys:
brand_name
likelihood_of_success
reasoning
"""

In [2]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [3]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['brand_description'], template='You are a master branding consulatant who specializes in naming brands. You come up with catchy and memorable brand names.\n\nTake the brand description below delimited by triple backticks and use it to create the name for a brand.\n\nbrand description: ```{brand_description}```\n\nthen based on the description and you hot new brand name give the brand a score 1-10 for how likely it is to succeed.\n\nFormat the output as JSON with the following keys:\nbrand_name\nlikelihood_of_success\nreasoning\n')

In [4]:
branding_messages = prompt_template.format_messages( brand_description="a cool hip new sneaker brand aimed at rich kids")

In [5]:
branding_messages

[HumanMessage(content='You are a master branding consulatant who specializes in naming brands. You come up with catchy and memorable brand names.\n\nTake the brand description below delimited by triple backticks and use it to create the name for a brand.\n\nbrand description: ```a cool hip new sneaker brand aimed at rich kids```\n\nthen based on the description and you hot new brand name give the brand a score 1-10 for how likely it is to succeed.\n\nFormat the output as JSON with the following keys:\nbrand_name\nlikelihood_of_success\nreasoning\n')]

In [10]:
(branding_messages[0].content)

'You are a master branding consulatant who specializes in naming brands. You come up with catchy and memorable brand names.\n\nTake the brand description below delimited by triple backticks and use it to create the name for a brand.\n\nbrand description: ```a cool hip new sneaker brand aimed at rich kids```\n\nthen based on the description and you hot new brand name give the brand a score 1-10 for how likely it is to succeed.\n\nFormat the output as JSON with the following keys:\nbrand_name\nlikelihood_of_success\nreasoning\n'

In [12]:
import ollama

response = ollama.chat(model='llama2', messages=[{'role': 'user', 'content': branding_messages[0].content}])
print("Chat response:", response['message']['content'])

Chat response: {
"brand_name": "Sneaker Syndicate",
"likelihood_of_success": 8,
"reasoning": "The name 'Sneaker Syndicate' is catchy and memorable, which is crucial for a brand aimed at rich kids. The word 'syndicate' implies a sense of exclusivity and luxury, which aligns well with the target audience. Additionally, the alliteration of 'sneaker' and 'syndicate' creates a fun and playful tone that could appeal to the desired demographic. Overall, I believe this name has a high likelihood of success in the market."
}


### STRUCTURED OUTPUT PARSER

In [22]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

brand_name_schema = ResponseSchema(name="brand_name",
                             description="This is the name of the brand")

likelihood_of_success_schema = ResponseSchema(name="likelihood_of_success",
                                      description="This is an integer score between 1-10")

reasoning_schema = ResponseSchema(name="reasoning",
                                    description="This is the reasons for the score")

response_schemas = [brand_name_schema, 
                    likelihood_of_success_schema,
                    reasoning_schema]

In [23]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [24]:
format_instructions = output_parser.get_format_instructions()

In [25]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"brand_name": string  // This is the name of the brand
	"likelihood_of_success": string  // This is an integer score between 1-10
	"reasoning": string  // This is the reasons for the score
}
```


In [26]:
template_string = """You are a master branding consulatant who specializes in naming brands. \
You come up with catchy and memorable brand names.

Take the brand description below delimited by triple backticks and use it to create the name for a brand.

brand description: ```{brand_description}```

then based on the description and you hot new brand name give the brand a score 1-10 for how likely it is to succeed.

{format_instructions}
"""

In [28]:
prompt = ChatPromptTemplate.from_template(template=template_string)
messages = prompt.format_messages(brand_description="a new nerf gun aimed for teenagers",
                                format_instructions=format_instructions)

In [29]:
messages

[HumanMessage(content='You are a master branding consulatant who specializes in naming brands. You come up with catchy and memorable brand names.\n\nTake the brand description below delimited by triple backticks and use it to create the name for a brand.\n\nbrand description: ```a new nerf gun aimed for teenagers```\n\nthen based on the description and you hot new brand name give the brand a score 1-10 for how likely it is to succeed.\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"brand_name": string  // This is the name of the brand\n\t"likelihood_of_success": string  // This is an integer score between 1-10\n\t"reasoning": string  // This is the reasons for the score\n}\n```\n')]

In [30]:
messages[0].content

'You are a master branding consulatant who specializes in naming brands. You come up with catchy and memorable brand names.\n\nTake the brand description below delimited by triple backticks and use it to create the name for a brand.\n\nbrand description: ```a new nerf gun aimed for teenagers```\n\nthen based on the description and you hot new brand name give the brand a score 1-10 for how likely it is to succeed.\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"brand_name": string  // This is the name of the brand\n\t"likelihood_of_success": string  // This is an integer score between 1-10\n\t"reasoning": string  // This is the reasons for the score\n}\n```\n'

In [31]:
response = ollama.chat(model='llama2', messages=[{'role': 'user', 'content': messages[0].content}])
print("Chat response:", response['message']['content'])

Chat response: ```json
{
	"brand_name": "TeenStrike",
	"likelihood_of_success": 8,
	"reasoning": "The name 'TeenStrike' is catchy and memorable, which is important for a brand aimed at teenagers. It also conveys the idea of fun and excitement, which are key qualities that teenagers look for in products. Additionally, the name has a nice ring to it and is easy to pronounce, making it more likely to stick in customers' minds."
}
```


In [32]:
type(response['message']['content'])

str

In [33]:
response_as_dict = output_parser.parse(response['message']['content'])

In [34]:
response_as_dict

{'brand_name': 'TeenStrike',
 'likelihood_of_success': 8,
 'reasoning': "The name 'TeenStrike' is catchy and memorable, which is important for a brand aimed at teenagers. It also conveys the idea of fun and excitement, which are key qualities that teenagers look for in products. Additionally, the name has a nice ring to it and is easy to pronounce, making it more likely to stick in customers' minds."}

In [35]:
type(response_as_dict)

dict

In [36]:
response_as_dict["brand_name"]

'TeenStrike'